In [19]:
# Load all required libraries
import time
import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initiate web driver
driver = webdriver.Chrome()
url = 'https://www.finder.com.au/savings-accounts/best-savings-accounts'
driver.get(url)

# Scroll down-up to load elements
time.sleep(2.5)

driver.execute_script("window.scrollTo(0, 2080)")

time.sleep(2.5)

driver.execute_script("window.scrollTo(0, 1080)")

# Set values for today, affiliate, and rank
get_today = datetime.datetime.now()
today = get_today.strftime('%d/%m/%Y')
affiliate = 'Finder'
rank = 1

# Create list to be used as df column headers
results = [['Date', 'Affiliate', 'Position', 'Provider', 'Product', 'Maximum Rate', 'Standard Rate', 'Max Rate Conditions', 'Savings Details']]

# Get number of savings listing cards for range
savings = WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="comparison-table-0000000000"]/tbody//tr')))

# Click on all 'View Details' collapsible elements to load savings_details text element
for i in range(1, len(savings) + 1):
    viewClass = f'/html/body/div[1]/div/div[5]/div[2]/div[2]/div[4]/section/form/table/tbody/tr[{i}]/td[8]/div//a//span'
    driver.execute_script('arguments[0].click();', WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, viewClass))))
    
# Additional loading time for slow-loading collapsible elements
time.sleep(2.5)

# Get savings listing values and push to results list
for i in range(1, len(savings) + 1):
    savings_listing = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]//a[1]'))).text.split(' ', 1)
    savings_max_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[3]/div//span'))).text
    savings_standard_rate = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[4]/div/span'))).text
#     savings_intro_period = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[5]/div'))).text
    savings_conditions = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]/td[7]/div/div[1]'))).text.replace('\n', ', ')
    
    savings_details = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, f'//*[@id="comparison-table-0000000000"]/tbody/tr[{i}]//p'))).text
    
    if savings_listing[0]=='Bank' :
        savings_entry = f'{today} | {affiliate} | {rank} | Bank of Queensland | Bank {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
    elif savings_listing[0]=='Commonwealth' :
        savings_entry = f'{today} | {affiliate} | {rank} | Commonwealth Bank | {savings_listing[0]} {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
    else :
        savings_entry = f'{today} | {affiliate} | {rank} | {savings_listing[0]} | {savings_listing[0]} {savings_listing[1]} | {savings_max_rate} | {savings_standard_rate} | {savings_conditions} | {savings_details}'
    rank = rank + 1
    results.append(savings_entry.split('|'))

# Close web driver
driver.quit()

# Convert results list to dictionary
df = pd.DataFrame(results[1:], columns=results[0]).to_dict('list')

# Convert df dictionary to dataframe
df2 = pd.DataFrame(df)

# Preview dataframe df2
print(df2)

# Create .csv file from dataframe df2
df2.to_csv('EDBfinder.csv', index=False, encoding='utf-8')

           Date Affiliate Position              Provider  \
0   25/12/2022    Finder        1                Virgin    
1   25/12/2022    Finder        2                   AMP    
2   25/12/2022    Finder        3                Virgin    
3   25/12/2022    Finder        4    Bank of Queensland    
4   25/12/2022    Finder        5                  HSBC    
5   25/12/2022    Finder        6    Bank of Queensland    
6   25/12/2022    Finder        7                  HSBC    
7   25/12/2022    Finder        8                 Ubank    
8   25/12/2022    Finder        9    Bank of Queensland    
9   25/12/2022    Finder       10              Rabobank    
10  25/12/2022    Finder       11                Virgin    
11  25/12/2022    Finder       12     Commonwealth Bank    

                                              Product Maximum Rate  \
0           Virgin Money Boost Saver (25+ year olds)        4.30%    
1    AMP Saver Account (Up to 4.10% from 1 Februar...       3.60%    
2        